In [265]:
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from selenium import webdriver
import html
from time import sleep
import urllib
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
from time import time

Функция для получения id книг автора

In [266]:
def get_id_list(author_url, driver):
    driver.get(author_url)
    items = driver.find_elements_by_class_name("count-result__value")
    cards_count = int(items[0].text)

    while len(driver.find_elements_by_class_name("product-card")) != cards_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        elem = driver.find_element_by_class_name('js__show-more-cards')
        if elem:
            try:
                elem.click()
            except:
                pass

    cards = driver.find_elements_by_class_name("product-card")
    ids = [int(card.get_attribute("data-product")) for card in cards]

    return ids

Функции для получения информационной таблички и разделения рейтинга и голосов соответственно

In [267]:
def get_table_dict(table):
    titles = [item.text.strip() for item in table.findAll('div', class_="product-prop__title")]
    values = [item.text.strip() for item in table.findAll('div', class_="product-prop__value")]
    return dict(zip(titles, values))

def get_rating(soup):
    rating = soup.find('div', class_='rating').text
    values = re.findall(r'\d+', rating)
    return values[0], values[1]

Функция парсинга странички с книгой по ее id

In [268]:
def extract_book_info(book_id):
    book_url = "https://www.chitai-gorod.ru/catalog/book/{}".format(book_id)
    book_html = requests.get(book_url).text.strip()
    soup = BeautifulSoup(book_html, 'html.parser')
    
    info = {}
    info['ID карточки'] = book_id
    info['Название'] = soup.find('h1', class_='product__title js-analytic-product-title').text.strip()
    info['Автор'] = soup.find('a', class_='link product__author').text.strip()
    info['Рейтинг'], info['Голоса'] = get_rating(soup)
    
    price = soup.find('div', class_="product__price")
    if price:
        info['Цена'] = price.text.strip()[0:-2]  
    
    props_dict = get_table_dict(soup.find('div', class_="product__props"))
    info.update(props_dict)
    
    image = soup.select_one('.product__image')
    if image:
        info['Обложка'] = image.find('img')['src']
        
    return info
    

In [269]:
url_template = "https://www.chitai-gorod.ru/search/result.php?q={}&type=author"
authors = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
books_ids = []
with webdriver.Safari() as driver:  
    for author in authors:
        url = url_template.format(urllib.parse.quote(author, encoding='windows-1251'))
        author_cards = get_id_list(url, driver)
        books_ids += author_cards
        print("{}: {}".format(author, len(author_cards)))

Фрай М.: 153
Хантер Э.: 97
Емец Д.: 192


In [271]:
%%time

t = ThreadPool(8)
result = t.map(extract_book_info, books_ids)
t.close()
t.join()

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

cols = df.columns.values
cols = np.delete(cols, np.argwhere(cols == 'Обложка'))
cols = np.append(cols, 'Обложка')
df = df[cols]

with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

CPU times: user 52.5 s, sys: 2.79 s, total: 55.3 s
Wall time: 1min 17s
